In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import mplfinance as mpf
import yfinance as yf
import time
#import talib as ta
from talib import abstract
import os
import matplotlib.pyplot as plt
import talib
import math
pd.set_option("display.max_rows", None, "display.max_columns", None)
from datetime import datetime



In [ ]:
#%%script False --no-raise-error
#!python3 'get_snp500.py'


In [ ]:
today=datetime.now().strftime("%m%d%y")
print(f"Today: {today}")
ch='AAPL'
tick="/home/thakur/test/snp500/{}.csv".format(ch)
df=pd.read_csv(tick)
df.rename(columns={'Date':'date','Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume'},inplace=True)
df.tail(5)

In [ ]:
# #sma=abstract.RSI(df)
# #help(abstract.RSI)
# atr=abstract.ATR(df.tail(20))
# atr.tail(15)


# #bband=abstract.BBANDS(df.tail(200))
# sd=1;period=14
# u,m,l=talib.BBANDS(df.close,timeperiod=period, nbdevup=sd, nbdevdn=sd)


# #u=u[-1:];m=m[-1:];l=l[-1:];w=(u-l)/m
# u=u.tail(1).values[0];m=m.tail(1).values[0];l=l.tail(1).values[0];w=(u-l)/m
# print(f"u: {u}")
# print(f"m: {m}")
# print(f"l: {l}")
# print(f"w: {w}")

In [ ]:
#dataframe with snp company list and other information
snp_df=pd.read_csv("/home/thakur/stock_information/snp500finalwvol.csv")
snp_df.head()


In [ ]:
#read the dataframe and calculate the values
csv_source="/home/thakur/test/snp500/"
st_list=snp_df['SYMBOL'].to_list()
print(st_list[:5])

In [ ]:
#check which dow jones ticker are in snp500
%%script False --no-raise-error
dow_list=pd.read_csv("/home/thakur/stock_information/dow.csv")['Symbol'].to_list()
dow_list
for c,i in enumerate(dow_list):
    if c%50==0:print(f"{c+1}:\n")
    if i in st_list:
        print(f"{i} in snp500")
    else:
        print("ALERT=====>")
        print(f"{i} not in snp500")

In [ ]:
def change_index(df):
    """Begins the index of df from 1"""
    df.index=np.arange(1,len(df.index)+1)
    return df

#create yfinance portfolio sample
def yahoo_portfolio(df,save_name):
    """
    returns the yfinance compatable portfolio template for given df name of save file
    """
    cols=['Symbol','Current Price','Date', 'Time', 'Change', 'Open', 'High', 'Low', 'Volume', 'Trade Date',
          'Purchase Price','Quantity','Commission','High Limit','Low Limit','Comment']

    sym=list(df.TICK)
    dic_val=[sym]+14*[None] #14 None
    my_dict=dict(zip(cols,dic_val)) #dictionary
    my_df=pd.DataFrame.from_dict(my_dict)
    my_df.to_csv(save_name+".csv")
    print(f"Created the yahoo template: {save_name}.csv from data frame !")
    
def plot_with_mpl(tick,root,day=60,sty='yahoo',typ='candle',ax=None):
    "returns the candle stick of a stock tick for the given root file."
    if ax is None:ax=plt.gca()
    df=pd.read_csv(root+tick+".csv").tail(day)
    df.Volume=df.Volume/10**5              #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    fig,ax=mpf.plot(df,type=typ,mav=(5,21),volume=True,tight_layout=False,figratio=(24,12),figscale=1,
                   returnfig=True,style=sty)
    
    ax[0].legend(["5-SMA","21-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
    ax[0].set_title(tick)
    #return ax
    #return fig
    
    mpf.show()

    
# def print_info(c,sector,industry,name,symbol,cp):
#     print(f"\n\n\nCOUNT   : {c+1}")
#     print(f"SECTOR  : {sector}\nINDUSTRY: {industry}\nNAME    : {name}\nSYMBOL  : {symbol}\nPRICE   : $ {cp}\n\n")
    
def print_info(c,sector,industry,name,symbol,cp,cng,cg):
    print(f"\n\n\nCOUNT   : {c+1}")
    print(f"SECTOR  : {sector}\nINDUSTRY: {industry}\nNAME    : {name}\nSYMBOL  : {symbol}\nPRICE   : $ {cp}\nCHNG%   : {cng}%\nCHG     : {cg}\n\n")
    
def style_negative(v, props=''):
    return props if v < 0 else None

k=1
def get_info(stock,par):
    """
    returns the par information regarding the stock
    """
    global k
    print(f"{k}/504");k+=1
    ticker=yf.Ticker(stock)
    print(f"Working for {stock}\n")
    return ticker.info[par]

def plot_all(df):
    for c,i in enumerate(df['TICKER'].tolist()):
        sector=df.loc[c+1,"SECTOR"];industry=df.loc[c+1,"SUBINDUSTRY"];cp=df.loc[c+1,"CP"];name=df.loc[c+1,"NAME"]
        cng=df.loc[c+1,"CHG%"];cg=df.loc[c+1,"CHG"]
        print_info(c,sector,industry,name,i,cp,cng,cg)
        time.sleep(1)
        plot_with_mpl(i,csv_source,50,'default')
        
#drop some columns
def drop_cols(df,cols=['HH','HL','HC','LH','LL','HV']):
    rest=df.drop(cols,axis=1)
    return rest

#get change
def get_change(cp,pct_change):
    """
    returns the change in the price based on the final price and %change
    """
    previous_price=(100*cp)/(100+pct_change)
    return round(cp-previous_price,2)

In [ ]:
#getting the function to plot all the plots as the subfigs
import math
def plot_all_subfig(df,root,typ='candle',sty='yahoo',day=60):
    """
    returns all the plots in a single file
    """
    rows=df.shape[0]
    per_row=3;cols=math.ceil(rows/per_row)
    fig,axs=plt.subplots(rows,cols)
    c=0 #counter
    all_ticks=df['TICKER'].tolist()
    for i in range(rows):
        
        for j in range(cols):
            tick=all_ticks[c]
            #pass
            #sector=df.loc[c+1,"SECTOR"];industry=df.loc[c+1,"SUBINDUSTRY"];cp=df.loc[c+1,"CP"];name=df.loc[c+1,"NAME"]
            #cng=df.loc[c+1,"CHG%"];tick=all_ticks[c]
            #print_info(c,sector,industry,name,all_ticks,cp,cng)
            #axs[i,j].plot_with_mpl(all_ticks,csv_source,50,'default')
            c+=1 #update counter
            df=pd.read_csv(root+tick+".csv").tail(day)
            df.Volume=df.Volume/10**5              #100k volume
            df.Date=pd.to_datetime(df.Date)
            df.set_index('Date',inplace=True)
            fig,ax=mpf.plot(df,type=typ,mav=(5,21),volume=True,tight_layout=False,figratio=(24,12),figscale=1,
                    returnfig=True,style=sty)
            ax[0].legend(["5-SMA","21-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
            ax[0].set_title(tick)
            return fig
            #return mpf
            #mpf.show()
            #return ax
    #return fig
    
    #mpf.show()

            
            
    
#     for c,i in enumerate(df['TICKER'].tolist()):
#         sector=df.loc[c+1,"SECTOR"];industry=df.loc[c+1,"SUBINDUSTRY"];cp=df.loc[c+1,"CP"];name=df.loc[c+1,"NAME"]
#         cng=df.loc[c+1,"%CHG"]
#         print_info(c,sector,industry,name,i,cp,cng)
#         plot_with_mpl(i,csv_source,50,'default')

In [ ]:
# def plot_with_mpl(tick,root,day=60,sty='yahoo',typ='candle'):
#     "returns the candle stick of a stock tick for the given root file."

#     df=pd.read_csv(root+tick+".csv").tail(day)
#     df.Volume=df.Volume/10**5              #100k volume
#     df.Date=pd.to_datetime(df.Date)
#     df.set_index('Date',inplace=True)
#     fig,axes=mpf.plot(df,type=typ,mav=(5,21),volume=True,tight_layout=False,figratio=(24,12),figscale=1,
#                     returnfig=True,style=sty)
#     axes[0].legend(["5-SMA","21-SMA"],loc='upper center')
#     #axes[0].legend(loc='upper left')
#     axes[0].set_title(tick)
#     mpf.show()

In [ ]:
#Reading the files
ticker_list=[]           #store ticker
current_price=[]         #current price

HH=[]                    #higher high
HL=[]                    #higher low
LH=[]                    #lower high
LL=[]                    #lower low
HC=[]                    #higher close
HV=[]                    #higher volume
rsi=[]                   #rsi list
sma_5=[]                 #sma_5 list
sma_10=[]                 #sma_10 list
sma_21=[]                 #sma_21 list
sma_50=[]                 #sma_50 list
sma_200=[]                 #sma_200 list
vol=[]                    #vol today
avgvol=[]                 #avg volume
pt_change=[]             #% change
tr=[]                   #atr
atr=[]                #average 10 day atr
sec_list=[]             #sector
ind_list=[]             #industry
bwidth=[]               #bollinger bandwith
d=3                     #days for hh etc.
j=1                     #counter
fil="s&p-500"

for f in st_list[:]:    #read these files
    #if f not in ['AAPL']:continue
    if(j-1)%50==0:print(f"{j}/{len(st_list)} {fil} : {f}\n")
    j+=1

    #print("source dir\t",csv_source)
    df=pd.read_csv(csv_source+f+".csv")
    df.fillna(0,inplace=True)
    
    

    #print the tail
    #print(f"Tail of {i}\n {df.tail(1)}")

    #ticker
    ticker_list.append(f)

    #current price 
    cp=round(df.tail(1)['Close'].values[0],2)
    #print(f"Current Price: {cp}")
    current_price.append(cp)

    #higher high
    hh=df.tail(d)['High'].is_monotonic_increasing
    #print(f"HH: {hh}\n")
    HH.append(hh)

    ##higher low
    hl=df.tail(d)['Low'].is_monotonic_increasing
    #print(f"HL: {hl}\n")
    HL.append(hl)

    #lower high
    lh=df.tail(d)['High'].is_monotonic_decreasing
    #print(f"LH: {lh}\n")
    LH.append(lh)

    ##lower low
    ll=df.tail(d)['Low'].is_monotonic_decreasing
    #print(f"LL: {ll}\n")
    LL.append(ll)

    #higher close
    hc=df.tail(d)['Close'].is_monotonic_increasing
    #print(f"HC: {hc}\n")
    HC.append(hc)

    #rsi
    rs=df.ta.rsi()[-1:].round(2).values[0]
    #print(f"rs\t{rs}")
    rsi.append(rs)

    #sma5
    sma5=round(df.tail(5)['Close'].mean(),2)
    #print(f"sma5: {sma5}")
    sma_5.append(sma5)
    
    #sma10
    sma10=round(df.tail(10)['Close'].mean(),2)
    #print(f"sma5: {sma5}")
    sma_10.append(sma10)

    #sma21
    sma21=round(df.tail(21)['Close'].mean(),2)
    #print(f"sma21: {sma21}")
    sma_21.append(sma21)

    #sma50
    sma50=round(df.tail(50)['Close'].mean(),2)
    #print(f"sma50: {sma50}")
    sma_50.append(sma50)

    #sma200
    sma200=round(df.tail(200)['Close'].mean(),2)
    #print(f"sma200: {sma200}")
    sma_200.append(sma200)

    #last volume
    v=(df.tail(1)['Volume'].values[0])/10**6
    v=round(v,2)
    vol.append(v)
    #print(f"today's volume {v} thousands")
    
    #higher volume
    hv=df.tail(d)['Volume'].is_monotonic_increasing
    #print(f"HC: {hc}\n")
    HV.append(hv)

    #last volume
    av=(df.tail(6).head(5)['Volume'].mean())/10**6  #avg volume for past5 days excluding today
    av=round(av,2)
    avgvol.append(av)
    #print(f"today's volume {av} thousands")

    #pct change
    pct=round((df.tail(2)['Close'].pct_change().values[1])*100,2)
    #print(f"%Change:\n {pct}")
    #pct=2
    pt_change.append(pct)
    #pct_change.append(pct)
    #print(f"%Change: {pct}")

    #atr value
    #df_20=df.tail(24) #for 10 day avg atr
    #changing the parameters for using abstract library
    df.rename(columns={'Date':'date','Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume'},inplace=True)
    atrr=abstract.ATR(df.tail(20))
    avgatr=atrr.tail(2).head(1).mean().round(2)
    #at=round(ta.atr(df_20['High'],df_20['Low'],df_20['Close']).values[-1],2)
    tran=abstract.TRANGE(df.tail(2)).mean().round(2)
    #print(f"atr: {at}")
    tr.append(tran)
    atr.append(avgatr)
    
    #getting the width of bband
    sd=1;period=14
    u,m,l=talib.BBANDS(df.close,timeperiod=period, nbdevup=sd, nbdevdn=sd)
    u=u.tail(1).values[0];m=m.tail(1).values[0];l=l.tail(1).values[0];w=round((u-l)/m,2)
    # print(f"u: {u}");print(f"m: {m}");print(f"l: {l}")
    # print(f"w: {w}")
    bwidth.append(w)
        
        
        
        
#create the data-frame
temp_df=pd.DataFrame({"TICKER":ticker_list,
                          "CP":current_price,
                          "HH":HH,
                          "HL":HL,
                          "LH":LH,
                          "LL":LL,
                          "HC":HC,
                          "HV":HV,
                          "RSI":rsi,
                          "SMA5":sma_5,
                          "SMA10":sma_10,
                          "SMA21":sma_21,
                          "SMA50":sma_50,
                          "SMA200":sma_200,
                          "VOL":vol,
                          "AVGVOL5":avgvol,
                          "CHG%":pt_change,
                          "ATR14":atr,
                          "TR":tr,
                          "W":bwidth
                         })

In [ ]:
test=temp_df.head()
test
plot_all_subfig(test,root="/home/thakur/test/snp500/")

In [ ]:
snp_df.head()

In [ ]:
# #store float volume in the list
# for c, i in enumerate(obj_list[:]):
#     print(f"{c+1}/{len(snp_list)}: {i}\n")
#     try:
#         float_volume=i.info["floatShares"]
#     except:
#         float_volume=0
    
#     print(f"Float shares:{float_volume}\n")
#     float_shares.append(float_volume)
    
    

#test.to_csv("snpwithvolume.csv",index=False)
# for c,i in enumerate(snp_list):
#     print(f"{c}: {i} => {float_shares[c
#len(float_shares)

#store float volume in the list
# for c, i in enumerate(left):
#     print(f"{c+1}/{len(left)}: {i}\n")
#     try:
#         float_volume=i.info["floatShares"]
#     except:
#         float_volume=0
    
#     print(f"Float shares:{float_volume}\n")
#     #float_shares.append(float_volume)
    
# f=['HII']
# for ticker in f:
#     tick = yf.Ticker(ticker)
    
#     print(f"float Volume of {ticker}: {tick.info['floatShares']}")
    

# JOINING TWO DATAFRAMES

In [ ]:
#combining the temp_df and snp_dfM
final=pd.concat([temp_df,snp_df],axis=1,join='inner',ignore_index=False)

final=final.drop(["SYMBOL"],axis=1)
final=final.sort_values(by=['CP'])
final=change_index(final)
print(f"Total length: {len(final.index)}\n")
final.head()

# Adding TURNOVER column with w.r.t. today's volume and total float

In [ ]:
final["TURN"]=round(final["VOL"]/final["FLOAT"]*100,2)
#new order
co=final.columns.to_list();c=co[:20]+co[23:]+co[20:23]
final=final[c]
final['CHG']=final.apply(lambda x: get_change(x['CP'],x['CHG%']),axis=1)
final.head()
#final_turn=final_turn.sort_values(by=["TURN"],inplace=True)
#final_turn.head(20)

# TURNING

In [ ]:
final_turn=final.sort_values(by=["TURN"],ascending=False)
final_turn=drop_cols(final_turn)
final_turn=change_index(final_turn)
final_turn.head(20)


In [ ]:
# test_df=final_turn.head(10)
# #test_df['CHG%']
# #test_df['chg']=test_df.apply(lambda x: get_change(x['CP'],x['%CHG']),axis=1)
# test_df['CHG']=test_df.apply(lambda x: get_change(x['CP'],x['CHG%']),axis=1)
# test_df


# LOW FLOATS SNP 500

In [ ]:
low_float=final.sort_values(by=["FLOAT"])
low_float.head(10)

# HIGHEST +VE CHANGE

In [ ]:
high_change=final.sort_values(by=['CHG%'],ascending=False)
high_change=change_index(high_change)
high_change.head(20)

# HIGHEST -VE CHANGE

In [ ]:

high_change[::-1].head(5)

# ATR14 % w.r.t. CP

In [ ]:
df_atr=final
df_atr['%']=round(100*df_atr['ATR14']/df_atr['CP'],2)
df_atr=df_atr.sort_values(by=['%'],ascending=False)
df_atr=drop_cols(df_atr)

df_atr=change_index(df_atr)

df_atr.head(20)

# HIGHEST VOLUME

In [ ]:
vol_highest=final.sort_values(by=['VOL'],ascending=False)
vol_highest=change_index(vol_highest)
vol_highest.head(20)

# 14D  ATR

In [ ]:
#see the changes in the prices for a given stock
#ch='TSLA'
import seaborn as sns
sns.set_theme()
%matplotlib qt
atr_list=df_atr.head(20)['TICKER'].to_list()
for c,ch in enumerate(atr_list[:10]):
    tick="/home/thakur/test/snp500/{}.csv".format(ch)
    df=pd.read_csv(tick).tail(10)
    df.rename(columns={'Date':'date','Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume'},inplace=True)

    #df.date=pd.to_datetime(df['date']);

    df.set_index(keys=df.date)
    df=df.set_index(df.date)
    df.volume=round(df.volume/10**6,2) #volume in millions
    df['h-l']=df['high']-df['low'];df['h-o']=df['high']-df['open'];df['h-c']=df.high-df.close
    df.drop(columns=['date'],inplace=True)
    df
    %matplotlib inline
    fig, axes = plt.subplots(nrows=2, ncols=2)

    df['volume'].plot.bar(ax=axes[0,0],figsize=(10,6),color='grey'); 
    axes[0,0].set_title('volume vs time');
    axes[0,0].set_ylabel('volume (million)')
    axes[0,0].axhline(y=df['volume'].mean(),color='red',linestyle="--")

    df['h-l'].plot.bar(ax=axes[0,1]); axes[0,1].set_title('h-l');axes[0,1].set_ylabel('high-low')
    axes[0,1].axhline(y=df['h-l'].mean(),color='blue',linestyle="--")

    df['h-o'].plot.bar(ax=axes[1,0]); axes[1,0].set_title('h-o');axes[1,0].set_ylabel('high-open')
    axes[1,0].axhline(y=df['h-o'].mean(),color='pink',linestyle="--")
    df['h-c'].plot.bar(ax=axes[1,1],color='k'); axes[1,1].set_title('h-c');axes[1,0].set_ylabel('high-close')
    axes[1,1].axhline(y=df['h-c'].mean(),color='green',linestyle="--")
    fig.tight_layout()

    fig.suptitle(str(c+1)+": "+ch+" ("+df_atr.loc[c+1,"NAME"]+"-"+df_atr.loc[c+1,"SECTOR"]+")", fontsize=10,color='red')
    plt.show() #this shows all the plots in the loop

# - *VOL > AVGVOL5*
# - *%CHG > 0*
# - *CP > SMA5*
# - *SMA5> SMA10*


In [ ]:
hh=final[final['HH']==True];
hl=final[final['HL']==True];
# hc=final[final['HC']==True];
# hv=final[final['HV']==True]
df_chg=final[(final['VOL']>final['AVGVOL5']) & (final['CHG%']>0) & (final['CP']>final['SMA5'])&(final['SMA5']>final['SMA10'])
             & (final['HH']==True) & (final['HL']==True)]
df_chg.sort_values(by=['CP'])
df_chg=change_index(df_chg)
print(f"Total Length: {len(df_chg.index)}")
df_chg.head(20)

In [ ]:
# df_chg=df_chg.head(10)
# plot_all(df_chg)
# df_chg=df_chg.head(10)
# plot_all_subfig(df_chg)

# - PRICE ABOVE SMA50 and SMA200
# - SMA50>SMA200

In [ ]:
f1=final['CP']>final['SMA50']
f2=final['CP']>final['SMA200']
f3=final['SMA50']>final['SMA200']
df_above=final[f1 & f2 &f3]
df_above=change_index(df_above)
print(f"Total: {len(df_above.index)}")
#df_above=df_above.sort_values(by=['CP'])
df_above.head(5)

# Sort by 5 Day Average Volume

In [ ]:
df_vol=final.sort_values(by=['AVGVOL5'],ascending=False)
df_vol=df_vol[df_vol['CHG%']>0]
df_vol=change_index(df_vol)
print(f"Total Length: {len(df_vol.index)}")
df_vol=df_vol.head(10)
df_vol

In [ ]:
#plot_all(df_vol)

In [ ]:
# df_atr_s = df_atr['%CHG'].style.applymap(style_negative, props='color:red;')\
#               .applymap(lambda v: 'opacity: 20%;' if (v < 0.3) and (v > -0.3) else None)
# df_atr_s
# df_atr.head(20).style.set_properties(**{'background-color': 'black',
#                            'color': 'lawngreen',
#                            'border-color': 'white'})

# Conditions
1. HH
2. HL
3. HC
4. CP>SMA50
5. SMA50>SMA200
5. RSI>40
6. VOL>AVGVOL5

In [ ]:
#df_hl=final[(final['HH']==True) & (final['HL']==True) &(final['HC']==True)]
df_hl=final[(final['HH']==True) & (final['HL']==True)  & (final['CP']>final['SMA50']) & 
            (final['CP']>final['SMA200'])& (final['RSI']>40) & 
            (final['HC']==True) & 
            (final['VOL']>final['AVGVOL5']) & (final['SMA50']>final['SMA200'])]# & (final['CP']>final['SMA200'])]
#df_hl=final[(final['CP']>final['SMA50']) & (final['CP']>final['SMA200'])]
#df_hl["SMA50-CP"]=df_hl["SMA50"]-df_hl["CP"]
df_hl.loc[:,"SMA50-CP"]=df_hl.loc[:,"SMA50"]-df_hl.loc[:,"CP"]
df_hl=df_hl.sort_values(by=["SMA50-CP"],ascending=False)
df_hl=change_index(df_hl)
print(f"Total: {len(df_hl.index)}")
df_hl.head(20)


In [ ]:
saving=df_hl['TICKER']
saving.to_csv('watch'+today+".csv",index=False)

In [ ]:
#file='watch'+today+'.csv'

#print(f"Today {file}")
#!cat file
!cat 'watch082022.csv'

In [ ]:
#plot_all(df_hl)

# Conditions
1. HH
2. HL
3. HC
4. CP>SMA50
5. RSI>40
6. VOL>AVGVOL5

In [ ]:
#df_hl=final[(final['HH']==True) & (final['HL']==True) &(final['HC']==True)]
df_hl=final[(final['HH']==True) & (final['HL']==True)  & (final['CP']>final['SMA50']) 
           # & (final['RSI']>40) 
            & (final['HC']==True) & (final['VOL']>final['AVGVOL5'])]# & (final['CP']>final['SMA200'])]
#df_hl=final[(final['CP']>final['SMA50']) & (final['CP']>final['SMA200'])]
#df_hl["SMA50-CP"]=df_hl["SMA50"]-df_hl["CP"]
df_hl.loc[:,"SMA50-CP"]=df_hl.loc[:,"SMA50"]-df_hl.loc[:,"CP"]
#df_hl=df_hl.sort_values(by=["SMA50-CP"],ascending=False)
df_hl=df_hl.sort_values(by=["CP"],ascending=True)
df_hl=change_index(df_hl)
print(f"Total: {len(df_hl.index)}")
df_hl.head(25)

In [ ]:
df_hl=df_hl.head()
df_hl
#plot_all(df_hl)

# ABOVE SMA50 AND SMA200

In [ ]:
p1=final['CP']>final['SMA50'];p2=final['CP']>final['SMA200'];
df_above=final[p1 & p2];df_above.sort_values(by=['CP']);df_above=change_index(df_above);print(f"Total Length: {len(df_above.index)}")
df_above.head()

In [ ]:
#plot_all(df_above)

In [ ]:
# #df_hl=final[(final['HH']==True) & (final['HL']==True) &(final['HC']==True)]
# df_hl=final[(final['HH']==True) & (final['HL']==True) & (final['HC']==True) & (final['CP']>final['SMA5']) & (final['RSI']>50)]
# #df_hl=final[(final['CP']>final['SMA50']) & (final['CP']>final['SMA200'])]
# df_hl=change_index(df_hl)
# df_hl

In [ ]:
# for c,i in enumerate(df_hl['TICKER'].tolist()):
#     sector=df_hl.loc[c+1,"SECTOR"];industry=df_hl.loc[c+1,"SUBINDUSTRY"];cp=df_hl.loc[c+1,"CP"];name=df_hl.loc[c+1,"NAME"]

#     print_info(c,sector,industry,name,i,cp)
#     plot_with_mpl(i,csv_source,50,'default')

# LH, LL, LC

In [ ]:
#df_hl=final[(final['HH']==True) & (final['HL']==True) &(final['HC']==True)]
df_ll=final[(final['LH']==True) & (final['LL']==True)  & (final['CP']<final['SMA200'])&(final['CP']<final['SMA50']) &(final['CP']<final['SMA5']) &(final['RSI']<40)]
#df_hl=final[(final['CP']>final['SMA50']) & (final['CP']>final['SMA200'])]
df_ll=change_index(df_ll)
df_ll

In [ ]:
%%script false --no-raise-error
for c,i in enumerate(df_ll['TICKER'].tolist()):
    sector=df_ll.loc[c+1,"SECTOR"];industry=df_ll.loc[c+1,"SUBINDUSTRY"];cp=df_ll.loc[c+1,"CP"];name=df_ll.loc[c+1,"NAME"]

    print_info(c,sector,industry,name,i,cp)
    plot_with_mpl(i,csv_source,50,'default')

In [ ]:
%%script false --no-raise-error
rsi_30=final[final['RSI']<30]
rsi_30

In [ ]:
%lsmagic
